In [11]:
import pandas as pd
from datetime import datetime

In [30]:
resigned = pd.read_csv("resigned.csv")
reasons = pd.read_csv("reasons.csv")

In [31]:
resigned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2764 entries, 0 to 2763
Data columns (total 2 columns):
Date of Leaving    2764 non-null object
Staff ID           2764 non-null int64
dtypes: int64(1), object(1)
memory usage: 43.3+ KB


In [32]:
reasons.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2527 entries, 0 to 2526
Data columns (total 31 columns):
Timestamp                                2527 non-null object
Number                                   2523 non-null float64
Cannot adapt to working environement     240 non-null object
Change of career line                    1090 non-null object
Difficulty with location                 115 non-null object
Expectations mismatched                  5 non-null object
Family reasons                           252 non-null object
Further studies                          518 non-null object
Health issues unrelated to work          284 non-null object
Health issues related to work            3 non-null object
Difficulty with job scope                72 non-null object
Lack of career development               19 non-null object
Long working hours                       716 non-null object
Low basic pay                            778 non-null object
No promotional prospects                 

Drop the duplicate Staff IDs:

In [33]:
resigned.drop_duplicates(subset='Staff ID', keep='last', inplace=True)
reasons.drop_duplicates(subset='Number', keep='last', inplace=True)

Data cleaning:

In [34]:
#Resigned data:
resigned['Date of Leaving'] = pd.to_datetime(resigned['Date of Leaving'], format='%d/%m/%y') + pd.offsets.MonthEnd(n=1)
resigned['Staff ID'] = resigned['Staff ID'].astype(int)

In [35]:
resigned.head()

,Date of Leaving,Staff ID
0,2014-01-31,6324
1,2014-01-31,50948
2,2014-01-31,39592
3,2014-01-31,58098
4,2014-01-31,58320


In [36]:
#Reasons data:
reasons.drop(['Timestamp'], axis=1, inplace=True)
reasons.replace("Yes", 1, inplace=True)
reasons['Difficulty with location/transport'] = reasons[["Difficulty with location", "Transport problems"]].max(axis=1)
reasons['Cannot adapt to working environment/working conditions'] = reasons[["Cannot adapt to working environement", "Working conditions"]].max(axis=1)
reasons['Health issues'] = reasons[["Health issues unrelated to work", "Health issues related to work"]].max(axis=1)
reasons['Lack of career development/promotion prospects'] = reasons[["Lack of career development", "No promotional prospects"]].max(axis=1)
reasons.drop(['Difficulty with location', 'Transport problems'], axis=1, inplace=True)
reasons.drop(['Cannot adapt to working environement', 'Working conditions'], axis=1, inplace=True)
reasons.drop(['Health issues unrelated to work', 'Health issues related to work'], axis=1, inplace=True)
reasons.drop(['Lack of career development', 'No promotional prospects'], axis=1, inplace=True)
reasons.rename(columns={'Difficulty with clients': 'Difficulties with clients', 'Difficulty with colleagues': 
                        'Difficulties with colleagues', 'Difficulty with supervisors': 'Difficulties with supervisors',
                       'Number': 'Staff ID'}, inplace=True)
reasons['Staff ID'].fillna(0, inplace=True)
reasons['Staff ID'] = reasons['Staff ID'].astype(int)
reasons_melted = pd.melt(reasons, id_vars=['Staff ID'])
reasons_melted.dropna(inplace=True)

In [37]:
reasons.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2508 entries, 0 to 2526
Data columns (total 26 columns):
Staff ID                                                  2508 non-null int64
Change of career line                                     1080 non-null float64
Expectations mismatched                                   5 non-null float64
Family reasons                                            251 non-null float64
Further studies                                           517 non-null float64
Difficulty with job scope                                 72 non-null float64
Long working hours                                        714 non-null float64
Low basic pay                                             773 non-null float64
Personal reasons                                          112 non-null float64
Received a better job offer                               303 non-null float64
Left for higher salary                                    58 non-null float64
Retirement/taking a break   

In [38]:
reasons_melted.tail()

,Staff ID,variable,value
62575,78670,Lack of career development/promotion prospects,1.0
62601,73605,Lack of career development/promotion prospects,1.0
62684,98849,Lack of career development/promotion prospects,1.0
62690,66385,Lack of career development/promotion prospects,1.0
62692,84063,Lack of career development/promotion prospects,1.0


Merge and write:

In [39]:
df2 = pd.merge(reasons_melted, resigned, how='left', on=['Staff ID'])
df2.dropna(inplace=True)
df2.drop(['value'], axis=1, inplace=True)
#df2['Date of Leaving'] = df2['Date of Leaving'] + pd.offsets.MonthEnd(n=1)
df2.to_csv("resigned_clean.csv",index=False)